In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchain-openai faiss-cpu

# tutorial here https://python.langchain.com/docs/tutorials/sql_qa/

In [10]:
import getpass
import os
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [11]:
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
    os.environ["LANGSMITH_TRACING"] = "true"

In [12]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///diary.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['diary_entries']


In [13]:
from sqlalchemy import create_engine  # Import create_engine
from sqlalchemy.orm import sessionmaker
from create_diary_schema import DiaryEntry

# Connect to the SQLite database
engine = create_engine("sqlite:///diary.db")
Session = sessionmaker(bind=engine)
session = Session()

# Query all diary entries
entries = session.query(DiaryEntry).all()
for entry in entries:
    print(f"Date: {entry.date}, Content: {entry.content[:50]}...")  # Print the first 50 characters

Date: 2023-03-28, Content: - i would like to spend a day more in my body and ...
Date: 2025-04-08, Content: Today is 8th of April 2025. I want to honor this d...
Date: 2023-09-12, Content: - value of the day. compassion.
- this morning i w...
Date: 2024-03-12, Content: - encuentro con el monje induista de bilbao.
- dic...
Date: 2024-01-17, Content: - en la meditacion. tension entre el hyperachiever...
Date: 2024-09-09, Content: - first day of coworking at commonshub. i met a gi...
Date: 2024-03-06, Content: - learning to see everything with curiosity
- me l...
Date: 2025-04-04, Content: I am somewhere on the 4th of April 2025 and it's l...
Date: 2025-03-28, Content: - Today is 28th of March 2025. It's in the morning...
Date: 2022-12-24, Content: - la pregunta inicial. what is the spiritualist mo...
Date: 2023-08-25, Content: - el tiempo invertido en cuidados personales. cuid...
Date: 2024-06-20, Content: - i know it's difficult to change our daily habits...
Date: 2025-03-08, Content: H

In [14]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [14]:


from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [15]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [16]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 2
for message in query_prompt_template.messages:
    message.pretty_print()

/home/manuel/Documents/ACCION_old/clients/yves_cariou_3.9/venv/lib/python3.9/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}
================================ Human Message =================================

Question: {input}


In [17]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [18]:
write_query({"question": "How many diary entries are there?"})

{'query': 'SELECT COUNT(*) AS total_entries FROM diary_entries;'}

In [19]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [20]:
execute_query({"query": "SELECT COUNT(*) AS total_entries FROM diary_entries;"})

{'result': '[(102,)]'}

In [21]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [ ]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

AttributeError: 'StateGraph' object has no attribute 'add_sequence'